In [1]:
import pandas as pd
import numpy as np

In [2]:
data_for = pd.read_csv('dimension_ukri_grants_for_270825.csv')

In [3]:
data_for['for_code'].isna().sum()

np.int64(3448)

In [4]:
data_for[['for_code', 'for_name', 'for_level']].drop_duplicates().sort_values('for_code')

,for_code,for_name,for_level
82,30.0,"Agricultural, Veterinary and Food Sciences",FoR 1
35,31.0,Biological Sciences,FoR 1
10,32.0,Biomedical and Clinical Sciences,FoR 1
4,33.0,Built Environment and Design,FoR 1
0,34.0,Chemical Sciences,FoR 1
...,...,...,...
953,5203.0,Clinical and Health Psychology,FoR 2
991,5204.0,Cognitive and Computational Psychology,FoR 2
10161,5205.0,Social and Personality Psychology,FoR 2
526,NaN,NaN,FoR 2


In [5]:
data_for['ukri_grant_id'].sample()


88226    2458650
Name: ukri_grant_id, dtype: object

In [6]:
data_for[data_for['ukri_grant_id']=='EP/Y029585/1']

,Unnamed: 0,dimensions_grant_id,ukri_grant_id,research_council,start_year,start_date,end_year,end_date,grant_title,funding_gbp,for_level,for_code,for_name
200567,200567,grant.13905327,EP/Y029585/1,UK Research and Innovation,2023,2023-12-01,2025.0,2025-11-30,Ecological and Evolutionary framework for the ...,200511.0,FoR 2,3207.0,Medical Microbiology
200568,200568,grant.13905327,EP/Y029585/1,UK Research and Innovation,2023,2023-12-01,2025.0,2025-11-30,Ecological and Evolutionary framework for the ...,200511.0,FoR 2,3107.0,Microbiology
200569,200569,grant.13905327,EP/Y029585/1,UK Research and Innovation,2023,2023-12-01,2025.0,2025-11-30,Ecological and Evolutionary framework for the ...,200511.0,FoR 1,31.0,Biological Sciences
200570,200570,grant.13905327,EP/Y029585/1,UK Research and Innovation,2023,2023-12-01,2025.0,2025-11-30,Ecological and Evolutionary framework for the ...,200511.0,FoR 1,32.0,Biomedical and Clinical Sciences


In [7]:
for_count = data_for.groupby(['ukri_grant_id', 'for_level'], as_index=False).agg({'for_code':'nunique'}).rename(columns={'for_code':'for_count'})


In [8]:
data_for_merge = pd.merge(data_for, for_count, on=['ukri_grant_id', 'for_level'])


In [9]:
data_for_merge.isna().sum()

print(len(data_for_merge['ukri_grant_id'].unique()))

data_for_merge[data_for_merge['for_code'].isna()]['ukri_grant_id'].unique() # 1724 grants with no FOR code

89292


array(['11440', '11475', '11476', ..., '2939465', '2941347', '2938324'],
      shape=(1724,), dtype=object)

In [10]:
1724*2

3448

In [11]:
data_for_merge.dropna(subset=['for_code'], inplace=True)

In [12]:
data_for_merge['start_date'].isna().sum()
data_for_merge['end_date'].isna().sum()

np.int64(600)

In [13]:
data_for_merge.dropna(subset=['start_date', 'end_date'], inplace=True)

In [14]:
data_for_merge['start_date'] = pd.to_datetime(data_for_merge['start_date'])
data_for_merge['end_date'] = pd.to_datetime(data_for_merge['end_date'])


In [15]:
data_for_merge['start_year'] = data_for_merge['start_date'].dt.year
data_for_merge['end_year'] = data_for_merge['end_date'].dt.year

In [16]:
data_for_merge['fraction'] = 1 / data_for_merge['for_count']

In [17]:
data_for_merge['funding_gbp'].isna().sum()

np.int64(98176)

In [18]:
data_for_merge.dropna(subset=['funding_gbp'], inplace=True)

In [19]:
data_for_merge['funding_frac'] = data_for_merge['funding_gbp'] * data_for_merge['fraction']

In [20]:
data_for_group = data_for_merge.groupby(['for_level', 'for_code', 'for_name', 'start_year'], as_index=False).agg({'ukri_grant_id':'nunique', 'funding_frac':'sum'}).rename(columns={'ukri_grant_id':'grant_count', 'funding_frac':'funding_total'})

In [21]:
data_for_group[['for_level', 'for_code']].value_counts()

data_for_group[data_for_group['for_level']=='FoR 1']['for_name'].unique()

array(['Agricultural, Veterinary and Food Sciences',
       'Biological Sciences', 'Biomedical and Clinical Sciences',
       'Built Environment and Design', 'Chemical Sciences',
       'Commerce, Management, Tourism and Services',
       'Creative Arts and Writing', 'Earth Sciences', 'Economics',
       'Education', 'Engineering', 'Environmental Sciences',
       'Health Sciences', 'History, Heritage and Archaeology',
       'Human Society', 'Information and Computing Sciences',
       'Language, Communication and Culture', 'Law and Legal Studies',
       'Mathematical Sciences', 'Philosophy and Religious Studies',
       'Physical Sciences', 'Psychology'], dtype=object)

In [22]:
data_for_group[data_for_group['for_level']=='FoR 1'].pivot(index=['for_level', 'for_code', 'for_name'], columns='start_year', values='funding_total').reset_index().to_csv('for1_funding_by_year.csv', index=False)

In [23]:
data_for_group[data_for_group['for_level']=='FoR 1'].pivot(index=['for_level', 'for_code', 'for_name'], columns='start_year', values='grant_count').reset_index().to_csv('for1_grants_by_year.csv', index=False)

In [24]:
data_for['ukri_grant_id'].nunique()

89292

In [25]:
data_for_merge['ukri_grant_id'].nunique()

50389

In [26]:
50389/89292

0.5643170720781256

In [31]:
data_for_merge[data_for_merge['for_name'].str.contains('Philosophy')].sort_values(by='funding_gbp', ascending=False).head(20)

,Unnamed: 0,dimensions_grant_id,ukri_grant_id,research_council,start_year,start_date,end_year,end_date,grant_title,funding_gbp,for_level,for_code,for_name,for_count,fraction,funding_frac
215040,215040,grant.14050778,EP/Z532861/1,Engineering and Physical Sciences Research Cou...,2024,2024-03-31,2029,2029-03-30,The Alan Turing Institute Core Grant,100000000.0,FoR 2,5003.0,Philosophy,1,1.0,100000000.0
215041,215041,grant.14050778,EP/Z532861/1,Engineering and Physical Sciences Research Cou...,2024,2024-03-31,2029,2029-03-30,The Alan Turing Institute Core Grant,100000000.0,FoR 1,50.0,Philosophy and Religious Studies,1,1.0,100000000.0
135399,135399,grant.12888042,EP/X527257/1,Engineering and Physical Sciences Research Cou...,2022,2022-03-31,2027,2027-03-30,Royce Phase 2,96000000.0,FoR 1,50.0,Philosophy and Religious Studies,2,0.5,48000000.0
175424,175424,grant.13252607,107522,UK Research and Innovation,2023,2023-03-31,2026,2026-03-30,The Faraday Institution Phase 2 (107522) - F/Y...,79706464.0,FoR 1,50.0,Philosophy and Religious Studies,2,0.5,39853232.0
91407,91407,grant.9530186,107140,Innovate UK,2020,2020-09-30,2026,2026-09-29,The Living Laboratory: driving economic growth...,38123332.0,FoR 1,50.0,Philosophy and Religious Studies,2,0.5,19061666.0
115141,115141,grant.9727190,107137,Innovate UK,2021,2021-03-31,2027,2027-03-30,MyWorld,29908140.0,FoR 1,50.0,Philosophy and Religious Studies,2,0.5,14954070.0
41536,41536,grant.8558159,EP/T012021/1,Engineering and Physical Sciences Research Cou...,2019,2019-06-30,2022,2022-03-30,Recurrent funding for the RFI 2019 to 2021,29642554.0,FoR 2,5002.0,History and Philosophy Of Specific Fields,1,1.0,29642554.0
41535,41535,grant.8558159,EP/T012021/1,Engineering and Physical Sciences Research Cou...,2019,2019-06-30,2022,2022-03-30,Recurrent funding for the RFI 2019 to 2021,29642554.0,FoR 1,50.0,Philosophy and Religious Studies,2,0.5,14821277.0
78011,78011,grant.9402336,ES/V002740/1,UK Research and Innovation,2020,2020-08-31,2026,2026-11-30,The Productivity Institute,26621454.0,FoR 2,3902.0,"Education Policy, Sociology and Philosophy",1,1.0,26621454.0
159305,159305,grant.13252605,640151,Innovate UK,2022,2022-03-31,2025,2025-03-30,EXE,24096414.0,FoR 1,50.0,Philosophy and Religious Studies,2,0.5,12048207.0
